In [15]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


df = pd.read_csv('../data/new/all_stocks_5yr.csv')
df

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
...,...,...,...,...,...,...,...
619035,2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS
619036,2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS
619037,2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS
619038,2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS


In [16]:
# Assuming 'df' is your stock dataset containing columns 'date', 'open', 'high', 'low', 'close', 'volume', 'Name'

# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Pivot data so that each stock is a column and dates are the index
df_pivoted = df.pivot_table(index='date', columns='Name', values='close')


df

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
...,...,...,...,...,...,...,...
619035,2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS
619036,2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS
619037,2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS
619038,2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS


In [17]:
# Download S&P 500 index data
index_data = yf.download('^GSPC', start='2013-02-08', end='2018-02-07')['Adj Close']


[*********************100%***********************]  1 of 1 completed


In [18]:
# Calculate daily returns for stocks
returns = df_pivoted.pct_change().dropna()

# Calculate daily returns for the S&P 500 index
index_returns = index_data.pct_change().dropna()


/var/folders/js/nb82jgfj26d2vt9cssgprd900000gn/T/ipykernel_77115/2114197082.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = df_pivoted.pct_change().dropna()


In [19]:
# Correlation of stock returns with the index returns
correlation_matrix = returns.corrwith(index_returns)


In [20]:
# Convert correlation matrix to covariance matrix (optional, but useful)
cov_matrix = returns.cov()

# Perform eigenvalue decomposition
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

# Sort eigenvalues and eigenvectors in descending order
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvalues_sorted = eigenvalues[sorted_indices]
eigenvectors_sorted = eigenvectors[:, sorted_indices]

# Get the first eigenvector, which represents the principal component
principal_component = eigenvectors_sorted[:, 0]


In [22]:
# Normalize the principal component to get the weights
weights = principal_component / np.sum(np.abs(principal_component))

# Create a DataFrame to display the weights of each stock
weights_df = pd.DataFrame(weights, index=returns.columns, columns=['Weight'])

# Display the top contributing stocks
weights_df.sort_values(by='Weight', ascending=False).head()


,Weight
Name,
DPS,0.000898
XL,0.000300
SO,0.000279
WYNN,0.000136
AEP,0.000125
